# 3.线性回归的简洁实现
## 3.1 导入模块

In [1]:
!pip install d2l

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils import data
from d2l import torch as d2l      


## 3.2 生成数据集


In [3]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 3.3 读取数据集


In [5]:
def load_array(data_arrays, batch_size, is_train=True): 
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.0565,  0.3121],
         [ 3.0707, -0.9777],
         [ 1.2180,  0.4691],
         [ 2.3194,  0.2533],
         [ 0.2771,  1.1273],
         [-1.2179,  0.3388],
         [ 0.8401, -0.0738],
         [ 1.6606,  0.3701],
         [-0.2262, -0.3793],
         [-0.3632, -0.9301]]),
 tensor([[ 3.2508],
         [13.6618],
         [ 5.0292],
         [ 7.9879],
         [ 0.9247],
         [ 0.6017],
         [ 6.1367],
         [ 6.2495],
         [ 5.0413],
         [ 6.6368]])]

## 3.4 定义模型
使用框架的预定义好的层。首先导入nn模块（神经网络的缩写），定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

## 3.5 初始化模型参数
指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏差参数将初始化为零。\
- net[0]选择网络中的第一个图层 
- weight.data和bias.data方法访问参数 
- 使用替换方法normal_和fill_来重写参数值



In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 3.6 定义损失函数

In [8]:
loss = nn.MSELoss()  # 均方误差

## 3.7 定义优化算法

In [10]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03) #要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

## 3.8 训练模型 

In [11]:
num_epochs = 3       # 迭代周期
for epoch in range(num_epochs): 
    for X, y in data_iter:      # 完整遍历一次数据集（train_data）,不停地从中获取一个小批量的输入X和相应的标签y
        l = loss(net(X) ,y)     # 通过调用net(X)生成预测并计算损失l（前向传播）
        trainer.zero_grad()     # 梯度清零
        l.backward()            # 通过进行反向传播来计算梯度
        trainer.step()          # 通过调用优化器来更新模型参数 
    l = loss(net(features), labels) 
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000241
epoch 2, loss 0.000103
epoch 3, loss 0.000102


比较生成数据集的真实参数和通过有限数据训练获得的模型参数，首先从net访问所需的层，然后读取该层的权重和偏置。

In [12]:
w = net[0].weight.data
print('w的估计误差:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差:', true_b - b)

w的估计误差： tensor([-0.0001,  0.0003])
b的估计误差： tensor([-0.0003])
